In [1]:
import numpy as np
import random
import math
from scipy.stats import multivariate_normal, norm
import derivatives as dev
from tqdm import tqdm
import matplotlib.pyplot as plt
import lambda_sampler as ls
import tau_sampler as ts
from scipy.stats import halfcauchy

In [2]:
extracted_coefficients_directory = '../../../data/commaai/extracted_coefficients/20201027_filtered_gaussian_resampled/'
B_zeta_path = str(extracted_coefficients_directory + 'Bzeta/B_zeta.npy')
beta_path = str(extracted_coefficients_directory + 'beta/beta.csv')
z_path = str(extracted_coefficients_directory + 'Bzeta/tr_labels.npy')
beta = np.genfromtxt(beta_path, delimiter=',')
B_zeta = np.load(B_zeta_path)
B_zeta = B_zeta.reshape(B_zeta.shape[0], beta.shape[0])
z = np.load(z_path) #[0:B_zeta.shape[0]]
n = B_zeta.shape[0]
p = B_zeta.shape[1]

Lambda_old = np.repeat(1,p)
BoB = B_zeta**2
log_tau_old = math.exp(1)
dS2_old, ddS2_old, S2_old, S_old = dev.generate_dS2_ddS2_S2_S(Lambda_old, BoB)

In [3]:
# initialize Lambda
betaBt = beta.dot(B_zeta.T)

In [6]:
num_iterations = 10000
lambdaacc = np.zeros((len(Lambda_old),num_iterations))
all_betas = []
all_lambdas = []
tauacc = []
all_taus = []
for j in tqdm(range(0, num_iterations)):
    
    beta = ts.sample_beta(p, B_zeta, Lambda_old, S_old, z)
    all_betas.append(beta)
    
    betaBt = beta.dot(B_zeta.T)
    
    # update tau
    log_tau_new, decision_criterion_tau = ts.sample_tau(log_tau_old, Lambda_old, p)
    # decision to accept/reject tau

    if math.log(random.random()) <= decision_criterion_tau:
        log_tau_old = log_tau_new
        tauacc.append(1)
        all_taus.append(log_tau_new)
    else:
        tauacc.append(0)
    
    # update lambda
    for m in range(0, len(Lambda_old)):
        log_sq_lambda_new, decision_criterion_lambda, dS2_new, ddS2_new, S2_new, S_new, ddlunew, Lambda_new = ls.sample_lambda(m, 
                                                                                               log_tau_old, 
                                                                                               Lambda_old, p, beta, 
                                                                                               B_zeta, dS2_old, ddS2_old, 
                                                                                              S2_old, S_old, z, BoB, betaBt)
        
        print(math.log(random.random()), decision_criterion_lambda)
        if math.log(random.random()) <= decision_criterion_lambda:
            #if m < p - 1:
                #Lambda_old = np.append(np.append(Lambda_old[0:m], math.exp(0.5*log_sq_lambda_new)), Lambda_new[m + 1:])
            Lambda_old = np.copy(Lambda_new)
            #elif m == p - 1:
            #    Lambda_old = np.append(Lambda_old[0:m],  math.exp(0.5*log_sq_lambda_new))
            lambdaacc[m,j] += 1
            all_lambdas.append(Lambda_old)
            dS2_old, ddS2_old, S2_old, S_old = dS2_new, ddS2_new, S2_new, S_new
        
        else:
            lambdaacc[m,j] += 0
            all_lambdas.append(Lambda_old) 

  0%|          | 0/10000 [00:00<?, ?it/s]

-1.4223529232021654 -6838.4761123880235
-1.0105110197133014 -197.89074137930402
-2.4302801565212784 -189169.05074069952
-0.22892629502300532 -164.0262095723067
-0.45394464244214117 -232382210.1409905
-2.063031763798913 -93985300133856.22
-0.07109189766302262 -6210.336731460164
-0.5059843891605902 -1854.865633319123


  0%|          | 1/10000 [00:01<3:55:54,  1.42s/it]

-1.0149388007819526 -1837609.6238551196
-0.1323982290884016 -2924972021549.96
-0.1021759482581188 -6930.959957059356
-2.2163456307091915 -191.26495977601235
-0.7518300443486922 -190238.93257115455
-0.8743470769366364 -146.62250582596175
-0.13202471103715682 -232439881.26836562
-0.3742832097201178 -94112461425837.88
-2.034445805958779 -6224.075298867661
-0.8567168850945239 -1775.576674761176
-0.14506937628283975 -1841226.916487258


  0%|          | 2/10000 [00:02<3:32:12,  1.27s/it]

...

  0%|          | 7/10000 [00:06<2:34:05,  1.08it/s]

-1.2800019926049186 -1841410.7134253886
-0.32842296994006903 -2933016852610.73
-0.055668600793523905 -7033.796811381559
-0.19661096414849438 -213.44786537671416


  0%|          | 7/10000 [00:07<2:48:52,  1.01s/it]

-0.8868992109433276 -189305.0821429093


KeyboardInterrupt: 

In [5]:
print('Acceptance rate for lambda_js: ' + str(np.sum(lambdaacc/num_iterations, axis = 1))
     + '\nAcceptance rate for tau: ' + str(np.mean(tauacc)))


Acceptance rate for lambda_js: [0.0001 0.0001 0.0001 0.0001 0.     0.     0.0001 0.0001 0.0001 0.    ]
Acceptance rate for tau: 0.9333333333333333


In [ ]:
lambdaacc[0]